In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras import Sequential,Model
from keras.layers import Conv1D,Input,Softmax,concatenate,InputLayer,Flatten,Dense,Activation
import tensorflow as tf

In [2]:
df = pd.read_csv('D:\\Career\\GradProj\\Datasets\\dataset4.csv')

In [3]:
df.drop(['Unnamed: 0','window number','7 x', '8 x', '9 x', '10 x', '11 x', '12 x', '7 y',
       '8 y', '9 y', '10 y', '11 y', '12 y', '7 z', '8 z', '9 z', '10 z',
       '11 z', '12 z'],axis = 1,inplace=True)

In [6]:
df_eng_feat = np.empty((1,5,42))

In [7]:
for i in range(0,len(df),75):
    df_eng_feat_window = np.array([[
        df.loc[i:i+75].mean(),
        df.loc[i:i+75].max(),
        df.loc[i:i+75].min(),
        df.loc[i:i+75].median(),
        df.loc[i:i+75].std()]])
    df_eng_feat = np.concatenate([df_eng_feat, df_eng_feat_window])

In [8]:
from keras import Sequential
from tcn import TCN

In [9]:
# Adding callbacks

# class CustomSaver(tf.keras.callbacks.Callback):
#     def on_epoch_end(self,model , epoch, logs={}):
#         if epoch == 5:  # or save after some epoch, each k-th epoch etc.
#             self.model.save(f"D:\\Career\\GradProj\\Model\\All_models\\{epoch}.hd5")

# saver = CustomSaver()
rlr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="loss",
    factor=0.6,
    patience=2,
    )

checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='D:\\Career\\GradProj\\Model\\Best_model_1.hdf5', verbose=0, save_best_only=True)

csv_logger = tf.keras.callbacks.CSVLogger("D:\\Career\\GradProj\\Model\\model_history_log.csv", append=True)

In [44]:
df_eng_feat.shape

(574, 4, 210)

In [40]:
df_eng_feat = np.asarray(df_eng_feat).reshape(-1,4,42*5)

In [45]:
model = Sequential(
            TCN(nb_filters=210, kernel_size=5, nb_stacks=1, dilations=(1, 2, 4, 8, 16), 
            padding='same', use_skip_connections=True, dropout_rate=.2, return_sequences=True,
            kernel_initializer='glorot_normal',input_shape=[4,210]))

In [46]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
                        loss=tf.keras.losses.MeanSquaredError(), 
                        metrics=['accuracy'])

In [47]:
# training
model.fit(  df_eng_feat, df_eng_feat,
            epochs=100,
            callbacks=[rlr,checkpointer,csv_logger],
            batch_size=32)

Epoch 1/100
18/18 [==============================] - 3s 45ms/step - loss: nan - accuracy: 0.0139 - lr: 0.0010
Epoch 2/100
18/18 [==============================] - 1s 41ms/step - loss: nan - accuracy: 0.0000e+00 - lr: 0.0010
Epoch 3/100
18/18 [==============================] - 1s 40ms/step - loss: nan - accuracy: 0.0000e+00 - lr: 6.0000e-04
Epoch 4/100
18/18 [==============================] - 1s 39ms/step - loss: nan - accuracy: 4.3554e-04 - lr: 6.0000e-04
Epoch 5/100
18/18 [==============================] - 1s 38ms/step - loss: nan - accuracy: 0.0000e+00 - lr: 3.6000e-04
Epoch 6/100
18/18 [==============================] - 1s 43ms/step - loss: nan - accuracy: 0.0000e+00 - lr: 3.6000e-04
Epoch 7/100
18/18 [==============================] - 1s 37ms/step - loss: nan - accuracy: 0.0017 - lr: 2.1600e-04
Epoch 8/100
18/18 [==============================] - 1s 42ms/step - loss: nan - accuracy: 0.0013 - lr: 2.1600e-04
Epoch 9/100
18/18 [==============================] - 1s 39ms/step - loss: na

KeyboardInterrupt: 